# import package

In [16]:
import pandas as pd
import numpy as np
import datetime
import json
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup

#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

# import datasets

## about Covid-19 case

In [2]:
def formthepath():
    to_day = datetime.date.today() + datetime.timedelta(-2)
    formatted_day= to_day.strftime('%m-%d-%Y') 
    return formatted_day

doc_day = formthepath()
case_path = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv".format(doc_day)
case_df = pd.read_csv(case_path)

state = 'Florida'
case_df = case_df.query('Province_State == @state').dropna().reset_index(drop = True)
case_df = case_df.rename(columns = {'Lat':'Latitude', 'Long_':'Longitude','Admin2': 'County'})
case_df

,FIPS,County,Province_State,Country_Region,Last_Update,Latitude,Longitude,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,12001.0,Alachua,Florida,US,2020-07-07 04:34:00,29.678665,-82.359282,1636,12,0,1624.0,"Alachua, Florida, US",608.081236,0.733496
1,12003.0,Baker,Florida,US,2020-07-07 04:34:00,30.330601,-82.284675,106,4,0,102.0,"Baker, Florida, US",362.889421,3.773585
2,12005.0,Bay,Florida,US,2020-07-07 04:34:00,30.265487,-85.621226,744,5,0,739.0,"Bay, Florida, US",425.860737,0.672043
3,12007.0,Bradford,Florida,US,2020-07-07 04:34:00,29.950797,-82.166116,95,2,0,93.0,"Bradford, Florida, US",336.867487,2.105263
4,12009.0,Brevard,Florida,US,2020-07-07 04:34:00,28.294095,-80.730910,2591,19,0,2572.0,"Brevard, Florida, US",430.440142,0.733308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,12125.0,Union,Florida,US,2020-07-07 04:34:00,30.044130,-82.374974,89,2,0,87.0,"Union, Florida, US",584.104483,2.247191
63,12127.0,Volusia,Florida,US,2020-07-07 04:34:00,29.058589,-81.182633,2804,58,0,2746.0,"Volusia, Florida, US",506.792172,2.068474
64,12129.0,Wakulla,Florida,US,2020-07-07 04:34:00,30.165489,-84.398952,90,1,0,89.0,"Wakulla, Florida, US",266.753609,1.111111
65,12131.0,Walton,Florida,US,2020-07-07 04:34:00,30.642040,-86.169357,324,9,0,315.0,"Walton, Florida, US",437.418153,2.777778


## about age

In [19]:
with urlopen('https://opendata.arcgis.com/datasets/a7887f1940b34bf5a02c6f7f27a5cb2c_0.geojson') as f:
    data = json.load(f)

#make empty dictionary and dataframe
to_make_dataframe = {}
df = pd.DataFrame(columns = data['features'][0]['properties'].keys())

#retrieve data from geojson documentation
for j in range(len(data['features'])):
    for k in data['features'][j]['properties'].keys():
            to_make_dataframe[k] = data['features'][j]['properties'][k]
    df_temp = pd.DataFrame([to_make_dataframe])
    df = pd.concat([df,df_temp],axis = 0,ignore_index=True)

#pay attention to the last row, the countyname is unknown, so I drop the last row
df.drop([67], inplace = True)
df = df.sort_values(by = 'COUNTY')

df.keys()

retrieve_list_1 = ['County_1','PUIsTotal', 'Age_0_4', 'Age_5_14', 'Age_15_24',
       'Age_25_34', 'Age_35_44', 'Age_45_54', 'Age_55_64', 'Age_65_74',
       'Age_75_84', 'Age_85plus','TPositive', 'TNegative','C_Female', 'C_Male',
       'C_SexUnkn', 'C_AllResTypes', 'C_Age_0_4', 'C_Age_5_14', 'C_Age_15_24',
       'C_Age_25_34', 'C_Age_35_44', 'C_Age_45_54', 'C_Age_55_64',
       'C_Age_65_74', 'C_Age_75_84', 'C_Age_85plus','T_total', 'T_negative', 'T_positive', 'Deaths']


retrieve_list_2 = ['County_1','TPositive', 'TNegative','C_Female', 'C_Male',
       'C_SexUnkn', 'C_AllResTypes', 'C_Age_0_4', 'C_Age_5_14', 'C_Age_15_24',
       'C_Age_25_34', 'C_Age_35_44', 'C_Age_45_54', 'C_Age_55_64',
       'C_Age_65_74', 'C_Age_75_84', 'C_Age_85plus','T_total', 'T_negative', 'T_positive', 'Deaths']


age_df = df.loc[:,retrieve_list_2].reset_index(drop = True).rename(columns = {'County_1': "County"})
age_df

,County,TPositive,TNegative,C_Female,C_Male,C_SexUnkn,C_AllResTypes,C_Age_0_4,C_Age_5_14,C_Age_15_24,C_Age_25_34,C_Age_35_44,C_Age_45_54,C_Age_55_64,C_Age_65_74,C_Age_75_84,C_Age_85plus,T_total,T_negative,T_positive,Deaths
0,Alachua,1792,14802,882,870,5,1757,14,41,614,430,188,182,146,111,42,22,47050,45247,1792,12
1,Baker,120,862,50,67,3,120,1,4,20,17,20,13,18,12,12,3,2121,2001,120,4
2,Bay,880,3012,442,401,2,845,13,18,158,193,159,138,104,60,29,8,9666,8762,880,5
3,Bradford,105,1424,71,34,0,105,1,6,13,17,16,12,9,8,12,11,2897,2790,105,2
4,Brevard,2813,18865,1437,1350,8,2795,37,76,603,596,431,397,343,179,91,59,54526,51704,2812,20
5,Broward,23783,89053,11851,11447,129,23427,397,877,3345,4875,4155,3879,3232,1643,916,454,240030,216103,23781,419
6,Calhoun,106,476,71,35,0,106,0,0,14,11,8,8,12,11,21,21,1027,918,106,6
7,Charlotte,975,4934,540,420,11,971,12,14,91,106,89,121,166,144,111,119,13990,13009,974,77
8,Citrus,457,3956,220,234,3,457,6,15,55,64,47,65,97,58,34,16,11396,10935,457,13
9,Clay,1158,7819,625,524,5,1154,7,31,160,194,176,197,159,123,65,45,19964,18793,1158,34


## about medical resources

In [4]:
ven_path = 'https://raw.githubusercontent.com/Archerontheway/Summer_C_project/master/Data%20Source/Raw_dataset/ventilator_data.csv'
doc_path = 'https://raw.githubusercontent.com/Archerontheway/Summer_C_project/master/Data%20Source/medical_resource_dataset/medical_resource.csv'

ven_df = pd.read_csv(ven_path)
doc_df = pd.read_csv(doc_path)

medical_df = pd.merge(doc_df,ven_df,how = 'left', on = 'County')
medical_df

,County,Clinician_Count_MD,Clinician_Count_DO,Nurse_practitioner_Count,Available beds,Bed Census,Total Staffed Bed Capacity,Est Available Ventilators,County ICU Beds,County Staffed Beds,Hospitalized,ICU Cases,Latest COVID-19 Cases,Percent Remaining Vent Capacity,Ventilator Capacity
0,Alachua,2225,74,600,267.0,1397.0,1664.0,110.0,387,"2,263",13.0,4.0,716.0,0.990692,430
1,Baker,17,2,17,95.0,530.0,625.0,3.0,4,81,1.0,1.0,41.0,0.750000,4
2,Bay,347,27,164,109.0,414.0,523.0,49.0,77,676,4.0,1.0,190.0,0.986711,75
3,Bradford,17,1,18,0.0,0.0,0.0,3.0,4,25,2.0,1.0,61.0,0.750000,4
4,Brevard,1239,118,421,410.0,1153.0,1563.0,125.0,288,"1,867",15.0,5.0,850.0,0.983607,305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Union,9,0,7,43.0,82.0,125.0,1.0,25,145,1.0,1.0,41.0,0.966942,30
63,Volusia,935,92,297,400.0,1038.0,1438.0,77.0,278,"1,514",21.0,7.0,1218.0,0.977671,314
64,Wakulla,11,2,9,0.0,0.0,0.0,NaN,NaN,NaN,1.0,1.0,40.0,0.000000,0
65,Walton,97,16,37,53.0,64.0,117.0,10.0,28,112,4.0,1.0,181.0,0.967742,31


## about population

In [5]:
pop_path = '/Users/chenhaoyi/JupyterNotebook/SummerProject/data/population_data/pop_data.xlsx'

pop_df = pd.read_excel(pop_path)
pop_df

,County,overweight or obese(%),underweight(%),Population,land area(sq mi),population density,RaceWhite,RaceBlack,RaceOther,RaceUnknown,HispanicYES,HispanicNO,HispanicUnk
0,Alachua,56.9,2.0,263148,875.02,300.733698,439,240,214,326,224,651,344
1,Baker,72.5,1.4,27785,585.23,47.477060,54,14,2,2,1,67,4
2,Bay,66.3,2.3,182482,758.46,240.595417,286,44,23,37,97,243,50
3,Bradford,70.6,2.7,26979,293.96,91.777793,64,17,3,0,6,76,2
4,Brevard,62.9,3.7,576808,1015.66,567.914460,1159,129,127,531,349,1064,533
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Union,78.6,1.6,15239,243.56,62.567745,38,21,2,1,3,57,2
63,Volusia,65.8,3.5,527634,1101.03,479.218550,986,329,404,483,262,1264,676
64,Wakulla,73.3,1.3,31877,606.42,52.565878,38,17,3,5,5,53,5
65,Walton,61.6,2.1,65858,1037.62,63.470249,197,12,5,10,63,150,11


## about economics

In [6]:
eco_path = '/Users/chenhaoyi/JupyterNotebook/SummerProject/data/economic_dataset/economic_data.xlsx'
eco_df = pd.read_excel(eco_path)

eco_df

,County,GDP,income,FPLi,cases,death,lethality,perin,logG,Logperin,Logl,population,pergdp
0,Alachua,12447381,11983398,97.45,551,10,0.018149,44390,7.095078,4.647285,-1.741152,267306,46.566037
1,Baker,595832,916242,96.45,30,4,0.133333,32313,5.775124,4.509377,-0.875061,28249,21.092145
2,Bay,7606775,8010201,95.83,153,4,0.026144,43231,6.881201,4.635795,-1.582631,167283,45.472493
3,Bradford,659865,930593,95.83,52,2,0.038462,33557,5.819455,4.525783,-1.414973,28682,23.006241
4,Brevard,23437467,27112075,98.36,565,15,0.026549,45425,7.369911,4.657295,-1.575957,594469,39.425886
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Union,299208,332065,94.61,38,1,0.026316,22227,5.475973,4.346881,-1.579784,15505,19.297517
63,Volusia,16446375,23201619,96.00,946,48,0.050740,42374,7.216070,4.627099,-1.294650,538763,30.526178
64,Wakulla,606163,1201906,94.02,35,1,0.028571,37026,5.782589,4.568507,-1.544068,32976,18.381944
65,Walton,2511146,4263054,97.37,150,9,0.060000,59728,6.399872,4.776178,-1.221849,70071,35.837165


## about mobility

In [7]:
data_path = '/Users/chenhaoyi/JupyterNotebook/SummerProject/data/Mobility_data/applemobilitytrends_2020_06_29.xlsx'
mobility_data = pd.read_excel(data_path)
county_data = mobility_data[mobility_data['county'].str.contains('County')].reset_index(drop = True)

mobility_df = county_data.drop(columns = ['geo_type','sub-region','country'],axis = 1)
mobility_df['county'] = mobility_df['county'].apply(lambda x: x[:-7])
mobility_df

,county,transportation_type,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,...,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29
0,Alachua,driving,100,102.21,111.36,116.55,141.97,130.21,92.69,104.06,...,121.64,87.92,105.58,104.22,107.49,115.48,127.04,122.16,95.71,104.81
1,Baker,driving,100,106.73,104.79,115.21,160.07,136.27,107.89,124.29,...,215.68,161.19,159.74,152.54,158.22,178.65,209.67,212.64,206.44,170.63
2,Bay,driving,100,106.55,107.45,109.01,120.89,110.74,84.16,103.14,...,431.91,324.65,337.99,326.81,330.84,352.98,391.02,410.83,327.23,320.18
3,Bradford,driving,100,101.71,96.89,118.15,141.98,123.76,92.72,107.28,...,168.79,124.68,136.89,137.74,131.42,141.68,177.30,164.72,126.87,136.31
4,Brevard,driving,100,98.62,101.06,104.98,121.63,118.77,93.47,102.19,...,137.21,106.54,121.69,120.04,123.34,127.72,143.87,132.59,106.95,117.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Taylor,driving,100,103.05,95.17,126.25,157.55,144.24,105.58,129.29,...,217.32,213.09,160.74,168.25,166.84,193.98,243.79,226.02,196.65,169.81
57,Volusia,driving,100,102.07,103.37,108.03,130.87,131.98,100.69,106.19,...,173.77,135.30,137.50,135.81,142.67,148.29,169.83,174.03,138.89,136.67
58,Wakulla,driving,100,103.25,104.06,114.88,134.91,154.80,106.72,118.13,...,239.65,173.30,165.72,151.87,157.33,169.60,198.92,220.66,182.00,156.88
59,Walton,driving,100,105.32,112.34,125.07,160.33,147.91,128.84,138.46,...,649.62,453.29,447.90,457.11,439.10,463.27,486.81,632.70,472.59,427.71


## Extra data from open health Florida

In [18]:
with urlopen('https://opendata.arcgis.com/datasets/a7887f1940b34bf5a02c6f7f27a5cb2c_0.geojson') as f:
    data = json.load(f)

#make empty dictionary and dataframe
to_make_dataframe = {}
df = pd.DataFrame(columns = data['features'][0]['properties'].keys())

#retrieve data from geojson documentation
for j in range(len(data['features'])):
    for k in data['features'][j]['properties'].keys():
            to_make_dataframe[k] = data['features'][j]['properties'][k]
    df_temp = pd.DataFrame([to_make_dataframe])
    df = pd.concat([df,df_temp],axis = 0,ignore_index=True)

#pay attention to the last row, the countyname is unknown, so I drop the last row
df.drop([67], inplace = True)
df = df.sort_values(by = 'COUNTY')
#df.keys()


retrieve_list = ['County_1','C_RaceWhite', 'C_RaceBlack',
       'C_RaceOther', 'C_RaceUnknown', 'C_HispanicYES', 'C_HispanicNO',
       'C_HispanicUnk', 'C_HospYes_Res',
       'C_HospYes_NonRes', 'C_NonResDeaths', 'C_FLResDeaths', 'CasesAll',
       'C_Men', 'C_Women', 'C_FLRes', 'C_NotFLRes', 'C_FLResOut', 'T_NegRes',
       'T_NegNotFLRes', 'T_total', 'T_negative', 'T_positive', 'Deaths']

opendata_df = df.loc[:,retrieve_list].reset_index(drop = True).rename(columns = {'County_1': "County"})

for i in range(len(opendata_df.keys())):
               if i == 0:
                   pass
               else:
                   opendata_df.iloc[:,i] = opendata_df.iloc[:,i].astype('float')
               
                            
opendata_df

,County,C_RaceWhite,C_RaceBlack,C_RaceOther,C_RaceUnknown,C_HispanicYES,C_HispanicNO,C_HispanicUnk,C_HospYes_Res,C_HospYes_NonRes,C_NonResDeaths,C_FLResDeaths,CasesAll,C_Men,C_Women,C_FLRes,C_NotFLRes,C_FLResOut,T_NegRes,T_NegNotFLRes,T_total,T_negative,T_positive,Deaths
0,Alachua,583.0,339.0,263.0,483.0,272.0,880.0,516.0,96.0,5.0,0.0,12.0,1701.0,846.0,850.0,1668.0,33.0,0.0,44152.0,250.0,46114.0,44402.0,1701.0,12.0
1,Baker,79.0,18.0,5.0,8.0,2.0,94.0,14.0,18.0,0.0,0.0,4.0,110.0,62.0,45.0,110.0,0.0,0.0,1909.0,60.0,2079.0,1969.0,110.0,4.0
2,Bay,519.0,100.0,42.0,122.0,142.0,492.0,149.0,38.0,2.0,2.0,5.0,818.0,385.0,431.0,783.0,35.0,0.0,8561.0,21.0,9414.0,8582.0,818.0,5.0
3,Bradford,74.0,24.0,3.0,1.0,6.0,90.0,6.0,14.0,0.0,0.0,2.0,102.0,35.0,67.0,102.0,0.0,0.0,2762.0,2.0,2868.0,2764.0,102.0,2.0
4,Brevard,1537.0,167.0,181.0,803.0,435.0,1443.0,810.0,137.0,3.0,0.0,20.0,2705.0,1315.0,1383.0,2688.0,17.0,0.0,50653.0,6.0,53374.0,50659.0,2705.0,20.0
5,Broward,9469.0,6438.0,1741.0,4597.0,7598.0,10262.0,4385.0,2317.0,60.0,22.0,418.0,22595.0,11128.0,11351.0,22245.0,350.0,0.0,209439.0,1261.0,233421.0,210700.0,22595.0,418.0
6,Calhoun,71.0,12.0,7.0,11.0,9.0,76.0,16.0,10.0,0.0,0.0,6.0,101.0,32.0,68.0,101.0,0.0,0.0,904.0,0.0,1008.0,904.0,101.0,6.0
7,Charlotte,716.0,69.0,17.0,139.0,65.0,706.0,170.0,159.0,1.0,0.0,77.0,944.0,409.0,526.0,941.0,3.0,0.0,12800.0,16.0,13766.0,12816.0,944.0,77.0
8,Citrus,322.0,23.0,19.0,55.0,34.0,326.0,59.0,46.0,0.0,0.0,13.0,419.0,212.0,205.0,419.0,0.0,0.0,10748.0,6.0,11177.0,10754.0,419.0,13.0
9,Clay,476.0,181.0,83.0,330.0,90.0,620.0,360.0,111.0,0.0,0.0,34.0,1074.0,485.0,584.0,1070.0,4.0,0.0,18465.0,18.0,19571.0,18483.0,1074.0,34.0


## create new variable

In [21]:
with urlopen('https://opendata.arcgis.com/datasets/a7887f1940b34bf5a02c6f7f27a5cb2c_0.geojson') as f:
    data = json.load(f)

#make empty dictionary and dataframe
to_make_dataframe = {}
df = pd.DataFrame(columns = data['features'][0]['properties'].keys())

#retrieve data from geojson documentation
for j in range(len(data['features'])):
    for k in data['features'][j]['properties'].keys():
            to_make_dataframe[k] = data['features'][j]['properties'][k]
    df_temp = pd.DataFrame([to_make_dataframe])
    df = pd.concat([df,df_temp],axis = 0,ignore_index=True)

#pay attention to the last row, the countyname is unknown, so I drop the last row
df.drop([67], inplace = True)
df = df.sort_values(by = 'COUNTY')

df.keys()

retrieve_list = ['County_1','TPositive', 'TNegative','C_Female', 'C_Male',
       'C_SexUnkn', 'C_AllResTypes', 'C_Age_0_4', 'C_Age_5_14', 'C_Age_15_24',
       'C_Age_25_34', 'C_Age_35_44', 'C_Age_45_54', 'C_Age_55_64',
       'C_Age_65_74', 'C_Age_75_84', 'C_Age_85plus','T_total', 'T_negative', 'T_positive', 'Deaths']

age_add_variable = df.loc[:,retrieve_list].reset_index(drop = True).rename(columns = {'County_1': "County"})
age_add_variable['lethality_ratio'] = (age_add_variable['Deaths'] / age_add_variable['T_positive'])*100
age_add_variable['C_plus_65'] = age_add_variable['C_Age_65_74'] + age_add_variable['C_Age_75_84'] + age_add_variable['C_Age_85plus']
age_add_variable['C_plus_65_ratio'] = (age_add_variable['C_plus_65'] / age_add_variable['T_positive']) * 100

for i in range(len(age_add_variable.keys())):
               if i == 0:
                   pass
               else:
                   age_add_variable.iloc[:,i] = age_add_variable.iloc[:,i].astype('float')
               
                            
age_add_variable

,County,TPositive,TNegative,C_Female,C_Male,C_SexUnkn,C_AllResTypes,C_Age_0_4,C_Age_5_14,C_Age_15_24,C_Age_25_34,C_Age_35_44,C_Age_45_54,C_Age_55_64,C_Age_65_74,C_Age_75_84,C_Age_85plus,T_total,T_negative,T_positive,Deaths,lethality_ratio,C_plus_65,C_plus_65_ratio
0,Alachua,1792.0,14802.0,882.0,870.0,5.0,1757.0,14.0,41.0,614.0,430.0,188.0,182.0,146.0,111.0,42.0,22.0,47050.0,45247.0,1792.0,12.0,0.669643,175.0,9.765625
1,Baker,120.0,862.0,50.0,67.0,3.0,120.0,1.0,4.0,20.0,17.0,20.0,13.0,18.0,12.0,12.0,3.0,2121.0,2001.0,120.0,4.0,3.333333,27.0,22.500000
2,Bay,880.0,3012.0,442.0,401.0,2.0,845.0,13.0,18.0,158.0,193.0,159.0,138.0,104.0,60.0,29.0,8.0,9666.0,8762.0,880.0,5.0,0.568182,97.0,11.022727
3,Bradford,105.0,1424.0,71.0,34.0,0.0,105.0,1.0,6.0,13.0,17.0,16.0,12.0,9.0,8.0,12.0,11.0,2897.0,2790.0,105.0,2.0,1.904762,31.0,29.523810
4,Brevard,2813.0,18865.0,1437.0,1350.0,8.0,2795.0,37.0,76.0,603.0,596.0,431.0,397.0,343.0,179.0,91.0,59.0,54526.0,51704.0,2812.0,20.0,0.711238,329.0,11.699858
5,Broward,23783.0,89053.0,11851.0,11447.0,129.0,23427.0,397.0,877.0,3345.0,4875.0,4155.0,3879.0,3232.0,1643.0,916.0,454.0,240030.0,216103.0,23781.0,419.0,1.761911,3013.0,12.669778
6,Calhoun,106.0,476.0,71.0,35.0,0.0,106.0,0.0,0.0,14.0,11.0,8.0,8.0,12.0,11.0,21.0,21.0,1027.0,918.0,106.0,6.0,5.660377,53.0,50.000000
7,Charlotte,975.0,4934.0,540.0,420.0,11.0,971.0,12.0,14.0,91.0,106.0,89.0,121.0,166.0,144.0,111.0,119.0,13990.0,13009.0,974.0,77.0,7.905544,374.0,38.398357
8,Citrus,457.0,3956.0,220.0,234.0,3.0,457.0,6.0,15.0,55.0,64.0,47.0,65.0,97.0,58.0,34.0,16.0,11396.0,10935.0,457.0,13.0,2.844639,108.0,23.632385
9,Clay,1158.0,7819.0,625.0,524.0,5.0,1154.0,7.0,31.0,160.0,194.0,176.0,197.0,159.0,123.0,65.0,45.0,19964.0,18793.0,1158.0,34.0,2.936097,233.0,20.120898


## extra data specific for age

In [26]:
extra_path = '/Users/chenhaoyi/JupyterNotebook/SummerProject/data/age_dataset/Provisional_COVID-19_Death_Counts_by_Sex__Age__and_State.csv'
extra_df = pd.read_csv(extra_path)

state = 'Florida'
extra_FL_df = extra_df.query('State == @state').loc[:,['Sex','Age group','COVID-19 Deaths','Total Deaths','Pneumonia Deaths','Pneumonia and COVID-19 Deaths','Influenza Deaths','Pneumonia, Influenza, or COVID-19 Deaths']].reset_index(drop = True)
extra_FL_df

,Sex,Age group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths"
0,Male,Under 1 year,0.0,275.0,NaN,0.0,0.0,NaN
1,Male,1-4 years,0.0,56.0,NaN,0.0,NaN,NaN
2,Male,5-14 years,0.0,86.0,NaN,0.0,NaN,NaN
3,Male,15-24 years,0.0,645.0,10.0,0.0,NaN,13.0
4,Male,25-34 years,10.0,1340.0,30.0,NaN,NaN,40.0
5,Male,35-44 years,40.0,1733.0,70.0,19.0,NaN,97.0
6,Male,45-54 years,85.0,2971.0,177.0,47.0,20.0,233.0
7,Male,55-64 years,192.0,6914.0,475.0,114.0,31.0,584.0
8,Male,65-74 years,379.0,10529.0,950.0,203.0,36.0,1162.0
9,Male,75-84 years,502.0,12656.0,1236.0,270.0,34.0,1501.0
